In [1]:
%matplotlib inline
import numpy as np
import matplotlib
matplotlib.rcParams['figure.figsize'] = (16.0,16.0)
import pandas as pd


## narrowPeak/broadPeak

**chrom** - Name of the chromosome (or contig, scaffold, etc.).

**chromStart** - The starting position of the feature in the chromosome or scaffold. The first base in a chromosome is numbered 0.

**chromEnd** - The ending position of the feature in the chromosome or scaffold. The chromEnd base is not included in the display of the feature. For example, the first 100 bases of a chromosome are defined as chromStart=0, chromEnd=100, and span the bases numbered 0-99.

**name** - Name given to a region (preferably unique). Use '.' if no name is assigned.

**score** - Indicates how dark the peak will be displayed in the browser (0-1000). If all scores were '0' when the data were submitted to the DCC, the DCC assigned scores 1-1000 based on signal value. Ideally the average signalValue per base spread is between 100-1000.

**strand** - +/- to denote strand or orientation (whenever applicable). Use '.' if no orientation is assigned.

**signalValue** - Measurement of overall (usually, average) enrichment for the region.

**pValue** - Measurement of statistical significance (-log10). Use -1 if no pValue is assigned.

**qValue** - Measurement of statistical significance using false discovery rate (-log10). Use -1 if no qValue is assigned.

### only in narrowPeak
**peak** - Point-source called for this peak; 0-based offset from chromStart. Use -1 if no point-source called.

In [2]:
narrowpeak_cols = ['chrom', 'chromStart', 'chromEnd', 'name', 'score', 'strand', 'signalValue', 'pValue', 'qValue', 'peak']
broadpeak_cols = narrowpeak_cols[:-1]
questpeak_cols = ['chrom', 'peak', 'signalValue']
columns = {'narrowPeak': narrowpeak_cols, 'broadPeak': broadpeak_cols, 'questPeak': questpeak_cols}

def determine_filetype(df):
    c = len(df.columns)
    if c==10:
        return 'narrowPeak'
    elif c==9:
        return 'broadPeak'
    elif c==3:
        return 'questPeak'
    return 'unknown'

replicate1 = pd.read_table('data/ENCSR000DPM/ENCFF001XLV.bed')
replicate2 = pd.read_table('data/ENCSR000DPM/ENCFF001XLW.bed')

replicate1_type = determine_filetype(replicate1)
assert replicate1_type!='unknown'

replicate2_type = determine_filetype(replicate2)
assert replicate2_type!='unknown'

replicate1.columns = columns[replicate1_type]
replicate2.columns = columns[replicate2_type]

In [3]:
def update_peaks(df):
    df['peak'] = np.where(df['peak'] == -1, (df['chromStart']+df['chromEnd'])/2.0, df['peak'])
    return df
replicate1 = update_peaks(replicate1)
replicate2 = update_peaks(replicate2)

In [4]:
replicate1['label'] = 1
replicate2['label'] = 2

In [41]:
combined_df = replicate1.append(replicate2)

In [42]:
columns_to_retain = ['chrom', 'peak', 'signalValue', 'label']
combined_df = combined_df[columns_to_retain]

In [43]:
combined_df = combined_df.sort(['chrom', 'peak'])

In [44]:
grouped = combined_df.groupby('chrom')

In [52]:
BIN_WIDTH = 40
df1 = combined_df[combined_df['chrom']=='chr1']
df1['bin'] = df1['peak']//BIN_WIDTH
unique_bins = np.unique(df1['bin'])
result_df = pd.DataFrame(columns=['bin', 'Replicate1 Peak', 'Replicate2 peak'])
l=0
for bin in unique_bins:
    df = df1[df1['bin']==bin]
    label = df['label']
    if len(label)>=2:
        #print('bin: {} replicate1_peak: {} replicate2_peak: {}'.format(bin, df[df['label']==1]['peak'].values, df[df['label']==2]['peak'].values))
        result_df.loc[l] = [bin, str(df[df['label']==1]['peak'].values),  str(df[df['label']==2]['peak'].values)]
        l=l+1

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [50]:
for name,grp in grouped:
    grp['bins'] = grp['peak']//BIN_WIDTH
    unique_bins = np.unique(bins)
    for bin in unique_bins:
        df = df1[df1['bin']==bin]
        label = df['label']

chr1
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr2
chr20
chr21
chr22
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chrX
chrY


In [19]:
unique_bins

array([  3.36000000e+02,   3.62000000e+02,   4.70000000e+02, ...,
         6.22919400e+06,   6.23001600e+06,   6.23047700e+06])

In [20]:
len(combined_df['bin'])

64393

In [54]:
print(result_df)

          bin     Replicate1 Peak     Replicate2 peak
0       21414          [ 856595.]          [ 856595.]
1       23870          [ 954815.]          [ 954815.]
2       29156         [ 1166255.]         [ 1166275.]
3       30013         [ 1200555.]         [ 1200555.]
4       30643         [ 1225735.]         [ 1225735.]
5       30685         [ 1227435.]         [ 1227415.]
6       30889         [ 1235595.]         [ 1235575.]
7       30937         [ 1237495.]         [ 1237495.]
8       31040         [ 1241615.]         [ 1241615.]
9       34410         [ 1376435.]         [ 1376435.]
10      43017         [ 1720695.]         [ 1720695.]
11      46013         [ 1840555.]         [ 1840555.]
12      46889         [ 1875595.]         [ 1875575.]
13      47291         [ 1891655.]         [ 1891675.]
14      49411         [ 1976475.]         [ 1976455.]
15      51595         [ 2063835.]         [ 2063835.]
16      51788         [ 2071535.]         [ 2071555.]
17      52647         [ 2105